Please provide the folder containing the soi.dat and recruit.dat datasets:

In [1]:
datafolder = "../data"

Let me check if you provided the correct folder:

In [2]:
import os
has_soi = sum([name.endswith("soi.dat") for name in os.listdir(datafolder)])
has_recruit = sum([name.endswith("recruit.dat") for name in os.listdir(datafolder)])

In [3]:
if (has_soi and has_recruit):
    print 'You are ready to go'
else:
    print 'Found files:'
    print os.listdir(datafolder)
    print ''
if not has_soi:
    print 'You are missing soi.dat'
if not has_recruit:
    print 'You are missing recruit.dat'

You are ready to go


## Setup
The following blocks setup some common functions and load the data.
You can skip it for now:

In [10]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
%load_ext autoreload
%autoreload 1
%aimport bdranalytics
import pandas as pd
import numpy as np
import os
import scipy as sc
from scipy.ndimage.interpolation import shift
import sklearn
from sklearn import linear_model, model_selection # in old versions use cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import make_scorer, r2_score
from sklearn.metrics.scorer import r2_scorer, mean_squared_error_scorer
import statsmodels
from statsmodels.tsa.api import VAR
from bdranalytics.model_selection.growingwindow import GrowingWindow
from IPython.display import display
import IPython
print "IPython version: {}".format(IPython.__version__)
print "statsmodels: {}".format(statsmodels.__version__)
print "numpy: {}".format(np.__version__)
print "scipy: {}".format(sc.__version__)
print "sklearn: {}".format(sklearn.__version__)
print "pandas: {}".format(pd.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
IPython version: 5.1.0
statsmodels: 0.6.1
numpy: 1.11.2
scipy: 0.18.1
sklearn: 0.18.1
pandas: 0.19.1


In [5]:
X_orig = pd.read_csv(os.path.join(datafolder, "soi.dat"), header=0, names=["soi"])
rng=pd.date_range('1/1/1866', periods=X_orig.size, freq='MS')
X_orig = X_orig.set_index(rng)
y_orig = pd.read_csv(os.path.join(datafolder, "recruit.dat"), header=0, names=["recruit"]).set_index(rng).iloc[:,0]

## Some data details

In [6]:
print X_orig.shape
print y_orig.shape
print X_orig.join(y_orig).head()
X_orig.join(y_orig).describe()

(452, 1)
(452,)
                       soi  recruit
1866-01-01 00:00:00  0.246    68.63
1866-02-01 00:00:00  0.311    68.63
1866-03-01 00:00:00  0.104    68.63
1866-04-01 00:00:00 -0.016    68.63
1866-05-01 00:00:00  0.235    68.63


,soi,recruit
count,452.000000,452.000000
mean,0.079381,62.248695
std,0.382915,28.006504
min,-1.000000,1.720000
25%,-0.182750,39.597500
50%,0.115000,68.625000
75%,0.366000,86.860000
max,1.000000,100.000000


## Exercise 1A : Creating features
Now we have 'X_orig' and 'y_orig', please create a new X dataframe, and y series.
The X dataframe should contain all features used to predict, while the y series should be the target variable.
If you use a lagged y to predict y, please use at least lag 1.

Documentation details:

  * http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shift.html
  * http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html


### Provided answer:

In [11]:
from bdranalytics.pandaspipeline.transformers import PdFeatureChain, PdFeatureUnion, PdWindowTransformer, PdLagTransformer

display(PdWindowTransformer(2, lambda x:x.max()).fit_transform(X_orig).head())
display(PdLagTransformer(2).fit_transform(y_orig.to_frame()).head())
display(PdFeatureUnion([
        PdWindowTransformer(2, lambda x:x.max()),
        PdLagTransformer(2)]
                    ).fit_transform(X_orig).head())
display(PdFeatureChain([PdWindowTransformer(2, lambda x:x.max()), PdLagTransformer(2)]).fit_transform(X_orig).head())

,soi_window2
1866-01-01 00:00:00,NaN
1866-02-01 00:00:00,0.311
1866-03-01 00:00:00,0.311
1866-04-01 00:00:00,0.104
1866-05-01 00:00:00,0.235


,recruit_lag2
1866-01-01 00:00:00,NaN
1866-02-01 00:00:00,NaN
1866-03-01 00:00:00,68.63
1866-04-01 00:00:00,68.63
1866-05-01 00:00:00,68.63


NameError: global name 'pd' is not defined

In [ ]:
window_transformers = PdFeatureUnion([PdWindowTransformer(window, lambda x: x.mean()) for window in range(1,12)])
lag_transformers = PdFeatureUnion([PdLagTransformer(lag) for lag in range(20)])
combined_features = PdFeatureChain([
        window_transformers,
        lag_transformers
        ])

In [ ]:
X = pd.concat([X_orig,  # the original features
               PdLagTransformer(1).fit_transform(y_orig.to_frame()) # the target to predict, lagged by 1
              ], axis=1, join_axes=[X_orig.index])
X = combined_features.fit_transform(X).dropna() # then wide range of possible windows & lags for all
y = y_orig[X.index] # because of dropped rows in X, need to also select corresponding remaining rows from y

## Testing your model
With a dataset of features (X), and a target variable (y), let's see how well we can predict the recruit.

In [ ]:
model_score = mean_squared_error
model_scorer = make_scorer(mean_squared_error, greater_is_better=False)

def cross_val(estimator, X, y, scorer = model_scorer, cv_count=10):
    return model_selection.cross_val_score(pipeline, X, y.to_frame(), 
                                           scoring = scorer,
                                           cv=GrowingWindow(cv_count))

def cross_val_train(estimator, X, y, scorer = model_scorer, cv_count=10):
    return [scorer(estimator.fit(X.iloc[train,:], y.iloc[train]),
                   X.iloc[train,:],
                   y.iloc[train])
             for train, test in GrowingWindow(cv_count).split(X)]

# if you don't have model_selection use the following:
def cross_val_for_old(estimator, X, y, scorer = model_scorer, cv_count=10):
    return [scorer(estimator.fit(X.iloc[train,:], y.iloc[train]),
                   X.iloc[test,:],
                   y.iloc[test])
             for train, test in GrowingWindow(cv_count).split(X)]

First we extract a train & test set from the full dataset

In [ ]:
i_train, i_test = list(itertools.islice(GrowingWindow(10).split(X), 8, 9))[0]
X_train = X.iloc[i_train,:]
y_train = y[i_train]

X_test = X.iloc[i_test,:]
y_test = y[i_test]
print "Train datasize = {}, Test datasets = {} ".format(X_train.shape, X_test.shape)
i_subtrain, i_subtest = list(itertools.islice(GrowingWindow(10).split(X_train), 8, 9))[0]

In [ ]:
display(X_train.columns)
display(y_train.head())
display(X_train.iloc[:,0:2].head())

In [ ]:
linear_regression = linear_model.LinearRegression()

pipeline = Pipeline([
        ("lm", linear_regression)
    ])

For reference, predicting y with the original features (just the soi):

In [ ]:
print "train:\t{}".format(np.mean(cross_val_train(pipeline, X_orig, y_orig, cv_count=10)))
print "test:\t{}".format(np.mean(cross_val(pipeline, X_orig, y_orig, cv_count=10)))

## The quality of your model:

Probably you do much better with your new set of features.

In [ ]:
print "train:\t{}".format(np.mean(cross_val_train(pipeline, X, y, cv_count=10)))
print "test:\t{}".format(np.mean(cross_val(pipeline, X, y, cv_count=10)))

## Exercise 1B: Choosing a different model
Feel free to select a different model, and see if you can increase the performance

In [ ]:
pipeline_ridge = Pipeline([
        ("ridge", Ridge(alpha=1.0))
    ])
print "train:\t{}".format(np.mean(cross_val_train(pipeline_ridge, X, y, cv_count=10)))
print "test:\t{}".format(np.mean(cross_val(pipeline_ridge, X, y, cv_count=10)))

In [ ]:
param_grid={'alpha':np.power(2.0,range(-2,2))}
param_grid

In [ ]:
ridge_cv = GridSearchCV(estimator = Ridge(),
                       param_grid=param_grid,
                       scoring=model_scorer,
                       n_jobs=1,
                       cv=GrowingWindow(10),
                       verbose=1).fit(X_train, y_train)

In [ ]:
ridge_cv.best_params_

In [ ]:
print "train:\t{}".format(np.mean(cross_val_train(ridge_cv.best_estimator_, X, y, cv_count=10)))
print "test:\t{}".format(np.mean(cross_val(ridge_cv.best_estimator_, X, y, cv_count=10)))

In [ ]:
param_grid={'alpha':np.power(2.0,range(-2,2)),
           'l1_ratio':[0.25,0.5,0.75]}
print param_grid
enet_cv = GridSearchCV(estimator = ElasticNet(),
                       param_grid=param_grid,
                       scoring=model_scorer,
                       n_jobs=1,
                       cv=GrowingWindow(10),
                       verbose=1).fit(X_train, y_train)
print enet_cv.best_params_

In [ ]:
print "train:\t{}".format(np.mean(cross_val_train(enet_cv.best_estimator_, X, y, cv_count=10)))
print "test:\t{}".format(np.mean(cross_val(enet_cv.best_estimator_, X, y, cv_count=10)))

## Exercise 2: Selecting features

Now try to select a subset of features, which results in a better results.

You can do it either manually (comparing coefficients / influences), or use feedback of RFE:  
  * http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

For the lm model, it's possible to get the coefficients as follows:

In [ ]:
print pd.Series(pipeline.fit(X_train, y_train).named_steps['lm'].coef_, X_train.columns).sort_values(ascending=False).head(5)
print pd.Series(pipeline.fit(X_train, y_train).named_steps['lm'].coef_, X_train.columns).sort_values(ascending=True).head(5)

You can also get the correlation coefficients from a VAR model

In [ ]:
var_model = VAR(X_orig.join(y_orig))
var_results = var_model.fit(6)
var_results.summary()

In [ ]:
var_results.plot_acorr()

In [ ]:
rfe = RFE(Ridge(alpha=1.0), step=5, n_features_to_select = 1)
np.mean(cross_val(rfe, X_train, y_train, cv_count=10))

In [ ]:
rfe_fit = RFE(Ridge(alpha=1.0), step=5, n_features_to_select = 1).fit(X_train, y_train)

In [ ]:
X_train.loc[:,rfe_fit.ranking_<=2].head()

In [ ]:
rfe_all = [np.mean(cross_val(
                                Ridge(alpha=1.0), 
                                X_train.loc[:,rfe_fit.ranking_<=i], 
                                y_train, 
                                cv_count=10))
     for i in range(1, max(rfe_fit.ranking_))]
best_index = np.array(rfe_all).argsort()[::-1][:1]
print 'Best nr of features = {}'.format(sum(rfe_fit.ranking_<=(best_index+1)))
print 'Which gives score   = {}'.format(rfe_all[best_index])

## Final score
Choose your best pipeline and test its performance:

### Provided solution:

In [ ]:
X_sub = X_train.loc[:, rfe_fit.ranking_<=(best_index+1)]
fit = Ridge(alpha=1.0).fit(X_sub, y_train)
test_predictions = fit.predict(X_test.loc[:, rfe_fit.ranking_ <= (best_index+1)])

### The score:

In [ ]:
-model_score(y_test, test_predictions)

In [ ]:
r2_score(y_test, test_predictions)